In [1]:
include("./MPS.jl")
using .MPSforQuantum
using TensorOperations
using LinearAlgebra

## MPOの用意

$Z_1 Z_2 + Z_2 Z_3 + Z_3 Z_4$

$$
\left[\begin{array}{c}
    Z_1 & I_1 & I_1\\
\end{array}\right]
\left[\begin{array}{c}
    Z_2 & 0 & 0\\
    0 & Z_2 & 0\\
    0 & 0 & I_2\\
\end{array}\right]
\left[\begin{array}{c}
    I_3 & 0 & 0\\
    0 & Z_3 & 0\\
    0 & 0 & Z_3\\
\end{array}\right]
\left[\begin{array}{c}
    I_4\\
    I_4\\
    Z_4
\end{array}\right] \quad
$$

In [2]:
pauliZ = convert(Array{ComplexF64,2}, [1 0; 0 -1])
pauliI = convert(Array{ComplexF64,2}, [1 0; 0 1])
zero = convert(Array{ComplexF64,2}, [0 0; 0 0])

O = []
push!(O, dstack((pauliZ, pauliI, pauliI)) )
push!(O, ddstack( [(pauliZ, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliI)] ))
push!(O, ddstack( [(pauliI, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliZ)] ))
push!(O, dstack((pauliI, pauliI, pauliZ)))
O[4]

2×2×3 Array{Complex{Float64},3}:
[:, :, 1] =
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

[:, :, 2] =
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

[:, :, 3] =
 1.0+0.0im   0.0+0.0im
 0.0+0.0im  -1.0+0.0im

In [3]:
N = 4
eps = 1e-3
D = 10

for i in 1:16
C0 = zeros(ComplexF64, 2^N)
C0[i] = 1 # '0001'
#C0 = normalize!(rand(ComplexF64, 2^N))
mps = MPS(C0, D)
# println(restore(mps, 4))
expc = expectation(mps, O)
println(expc)
end

3.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
-3.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
-3.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
-1.0 + 0.0im
1.0 + 0.0im
3.0 + 0.0im


## Right Normalize

In [4]:
N = 4
C0 = normalize!(rand(ComplexF64, 2^N))

D = 20
mps = MPS(C0, D, 'r') # convert to MPS
mps_size(mps) # print the size of MPS

expc = expectation(mps, O)
println("Initial Expectation Value: ", expc)

array 1's size: (1, 2)
array 2's size: (2, 4)
array 3's size: (4, 2)
array 4's size: (2, 1)
Num of parameters: 40
2^N: 16
Initial Expectation Value: -0.24536168433391928 + 8.673617379884035e-18im


In [5]:
pauliZ = convert(Array{ComplexF64,2}, [1 0; 0 -1])
pauliI = convert(Array{ComplexF64,2}, [1 0; 0 1])
zero = convert(Array{ComplexF64,2}, [0 0; 0 0])

O = []
push!(O, dstack((pauliZ, pauliI, pauliI)) )
push!(O, ddstack( [(pauliZ, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliI)] ))
push!(O, ddstack( [(pauliI, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliZ)] ))
push!(O, dstack((pauliI, pauliI, pauliZ)))

4-element Array{Any,1}:
 Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]

Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im]

Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im]
 Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -1.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im]
 Complex{Float64}[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im]

Complex{Float64}[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]

Complex{Float64}[0.0

### R expression が正しくできているか、期待値計算を使ってチェック

In [7]:
N = 4
D = 20
# C0 = normalize!(rand(ComplexF64, 2^N))

for i in 1:2^N
    C0 = zeros(ComplexF64, 2^N)
    C0[i] = 1 # computational basis
    mps = MPS(C0, D, 'r') # convert to MPS
    #mps_size(mps) # print the size of MPS
    R = R_expression(mps, O, 1)

    tmp_mps_cat = cat(mps[1][1], mps[1][2], dims=3)
    tmp_mps_dag_cat = cat(mps[1][1]', mps[1][2]', dims=3)
    tmp_O = O[1]
    res = zeros(ComplexF64, 1, 1)
    @tensor begin
        res[a0, a0_] = R[a1, b1, a1_] * tmp_mps_cat[a0_, a1_, sigma2] * tmp_O[sigma1, sigma2, b1] * tmp_mps_dag_cat[a1, a0, sigma1]
    end
    println(res, ": ", expectation(MPS(C0, D), O))
    
end

UndefVarError: UndefVarError: R_expression not defined

### L expression が正しくできているか、期待値計算を使ってチェック

In [55]:
function L_expression(mps::Array{Any,1}, O::Array{Any,1}, t::Int64)
        # t: target site, sites 1 to t-1 are merged to L expression
        N_site = size(mps)[1]
        n_phys = 2
        contracted_sites = []
        for i = 1:t-1
            if i==1
                tmp_mps = cat(mps[i][1][:], mps[i][2][:], dims=2)
                tmp_mps_dag = cat(mps[i][1]'[:], mps[i][2]'[:], dims=2)
                tmp_O = O[i]
                a_len = size(mps[i][1])[2]
                b_len = size(O[i][1, 1, :, :])[1]
                arr = zeros(ComplexF64, a_len, b_len, a_len)
                @tensor begin
                    arr[a0, b0, a0_] = tmp_mps_dag[a0, sigma1] * tmp_O[sigma1, sigma2, b0] * tmp_mps[a0_, sigma2]
                end
                push!(contracted_sites, arr)
            else
                tmp_mps = cat(mps[i][1], mps[i][2], dims=3)
                tmp_mps_dag = cat(mps[i][1]', mps[i][2]', dims=3)
                tmp_O = O[i]
                #tmp_site = contracted_sites[i-1]
                tmp_site = contracted_sites[1]
                a_len0 = size(tmp_site)[1]
                a_len1 = size(mps[i-1][1])[2]
                a_len2 = size(mps[i][1])[2]
                b_len = size(O[i][1, 1, :, :])[1]
                arr_0 = zeros(ComplexF64, n_phys, a_len1, b_len, a_len2)
                arr_1 = zeros(ComplexF64, n_phys, n_phys, a_len2, b_len, a_len2)
                arr = zeros(ComplexF64, a_len2, b_len, a_len2)
                @tensor begin
                    arr_0[sigma2, a1, b1, a2_] = tmp_site[a1, b1, a1_] * tmp_mps[a1_, a2_, sigma2]
                    arr_1[sigma1, sigma2, a2, b1, a2_] = tmp_mps_dag[a2, a1, sigma1] * arr_0[sigma2, a1, b1, a2_]
                    arr[a2, b2, a2_] = arr_1[sigma1, sigma2, a2, b1, a2_] * tmp_O[sigma1, sigma2, b1, b2]
                end
                contracted_sites[1] = arr
            end
            #push!(contracted_sites, arr)
        end
        return contracted_sites[1]
    end

L_expression (generic function with 1 method)

In [56]:
N = 4
D = 20
# C0 = normalize!(rand(ComplexF64, 2^N))

for i in 1:2^N
    C0 = zeros(ComplexF64, 2^N)
    C0[i] = 1 # computational basis
    mps = MPS(C0, D, 'l') # convert to MPS
    #mps_size(mps) # print the size of MPS
    L = L_expression(mps, O, 4)

    tmp_mps_cat = cat(mps[4][1], mps[4][2], dims=3)
    tmp_mps_dag_cat = cat(mps[4][1]', mps[4][2]', dims=3)
    tmp_O = O[4]
    res = zeros(ComplexF64, 1, 1)
    @tensor begin
        res[a1, a1_] = L[a0, b1, a0_] * tmp_mps_cat[a0_, a1_, sigma2] * tmp_O[sigma1, sigma2, b1] * tmp_mps_dag_cat[a1, a0, sigma1]
    end
    println(res, ": ", expectation(MPS(C0, D), O))
    
end

Complex{Float64}[3.0 + 0.0im]: 3.0 + 0.0im
Complex{Float64}[1.0 + 0.0im]: 1.0 + 0.0im
Complex{Float64}[-1.0 + 0.0im]: -1.0 + 0.0im
Complex{Float64}[1.0 + 0.0im]: 1.0 + 0.0im
Complex{Float64}[-1.0 + 0.0im]: -1.0 + 0.0im
Complex{Float64}[-3.0 + 0.0im]: -3.0 + 0.0im
Complex{Float64}[-1.0 + 0.0im]: -1.0 + 0.0im
Complex{Float64}[1.0 + 0.0im]: 1.0 + 0.0im
Complex{Float64}[1.0 + 0.0im]: 1.0 + 0.0im
Complex{Float64}[-1.0 + 0.0im]: -1.0 + 0.0im
Complex{Float64}[-3.0 + 0.0im]: -3.0 + 0.0im
Complex{Float64}[-1.0 + 0.0im]: -1.0 + 0.0im
Complex{Float64}[1.0 + 0.0im]: 1.0 + 0.0im
Complex{Float64}[-1.0 + 0.0im]: -1.0 + 0.0im
Complex{Float64}[1.0 + 0.0im]: 1.0 + 0.0im
Complex{Float64}[3.0 + 0.0im]: 3.0 + 0.0im


### Iterative ground state search

MPSに含まれるsiteを掃引しながら各siteに注目した局所最適化を繰り返し、Hamiltonian(MPO)の期待値を最小化するようにMPSを更新する。  
MPSの1つのsiteのみを変数としたとき、Hamiltoninaの最小化問題は変数としたsiteに含まれる要素の二次最適化問題の形式をとる。  
よって最小化は極値を求めればよく、期待値が極値をとるようなsiteを求める問題は行列の固有値問題に帰着する。

結局固有値問題に帰着するのは通常の状態ベクトルを用いた計算と同じだが、MPSのIterative ground state searchで行う固有値問題で扱う行列のサイズはbonding dimensionの大きさでリミットされる、すなわち量子ビット数に対して指数関数的に増える困難は無い。

### step 1

site 1について期待値最小化

In [138]:
C0 = zeros(ComplexF64, 2^N)
C0[9] = 1 # computational basis
mps = MPS(C0, D, 'r') # convert to MPS
before = expectation(mps, O)

t = 1

R = R_expression(mps, O, t)
tmp_O = O[t]
H = zeros(ComplexF64, size(R, 1), size(R, 3), size(tmp_O, 1), size(tmp_O, 2))
@tensor begin
    # H[a0, a0_, sigma1, sigma2] = tmp_O[sigma1, sigma2, b0] * R[a0, b0, a0_]
    H[sigma1, a0, sigma2, a0_] = tmp_O[sigma1, sigma2, b0] * R[a0, b0, a0_]
end

H_ = zeros(ComplexF64, size(tmp_O, 1) * size(R, 1), size(tmp_O, 2) * size(R, 3))
for i=1:size(H, 1), j=1:size(H, 2), k=1:size(H, 3), l=1:size(H, 4)
    H_[(i-1)*size(H, 2) + j, (k-1)*size(H, 4) + l] = H[i,j,k,l]
end
v = eigvecs(H_)[:, 1]
λ = eigvals(H_)[1]

d = convert(Int64, size(v, 1) / 2)
mps[t][1][:] = v[1:d]
mps[t][2][:] = v[d+1:2*d]

after = expectation(mps, O)

print(before, '\n', after)

1.0 + 0.0im
-1.0 + 0.0im

### step 2-1

site t, site t+1 に対しSVDを行い、site tをleft normalizeする。

In [2]:
function left_norm_for_2_sites(mps::Array{Any,1}, t::Int64, D::Int64)
    # t: target, site to be left normalized
    mps_ = copy(mps)
    site_1 = cat(mps[t][1], mps[t][2], dims=1)
    site_2 = cat(mps[t+1][1], mps[t+1][2], dims=2)
    mixed_site = site_1 * site_2
    A, M = SVD_L(mixed_site, D)
    col = convert(Int64, size(A, 1) / 2)
    mps_[t] = [A[1:col, :], A[(col+1):col*2, :]]
    col2 = convert(Int64, size(M, 2) / 2)
    mps_[t+1] = [M[:, 1:col2], M[:, (col2+1):col2*2]]
    return mps_
end

left_norm_for_2_sites (generic function with 1 method)

操作の前後でMPSが表す状態に変化がないことを確認

In [140]:
mps_ = left_norm_for_2_sites(mps, 1, 10)
for i = 0:15
    orig = restore(mps, i)
    norm = restore(mps_, i)
    print(orig == norm, ' ')
end

true true true true true true true true true true true true true true true true 

### step 2-2

site 2(~N-1)について期待値最小化

In [141]:
t = 2

R = R_expression(mps, O, t)
L = L_expression(mps, O, t)
tmp_O = O[t]

H = zeros(ComplexF64, size(tmp_O, 1), size(L, 1), size(R, 1), size(tmp_O, 2), size(L, 3), size(R, 3))
@tensor begin
    H[sigma1, a0, a1, sigma2, a0_, a1_] = L[a0, b0, a0_] * tmp_O[sigma1, sigma2, b0, b1] * R[a1, b1, a1_]
end

H_ = zeros(ComplexF64, size(tmp_O, 1) * size(L, 1) * size(R, 1), size(tmp_O, 2) * size(L, 3) * size(R, 3))
for i=1:size(H, 1), j=1:size(H, 2), k=1:size(H, 3), l=1:size(H, 4), m=1:size(H, 5), n=1:size(H, 6)
    H_[(i-1)*size(H, 2)*size(H, 3) + (j-1)*size(H, 3) + k, (l-1)*size(H, 5)*size(H, 6) + (m-1)*size(H, 6) + n] = H[i,j,k,l,m,n]
end
v = eigvecs(H_)[:, 1]
λ = eigvals(H_)[1]

d = convert(Int64, size(v, 1) / 2)

M_1 = transpose(reshape(v[1:d], size(transpose(mps[t][1]))))
M_2 = transpose(reshape(v[d+1:2*d], size(transpose(mps[t][1]))))
mps[t][1][:, :] = M_1
mps[t][2][:, :] = M_2

after = expectation(mps, O)

-3.0 + 0.0im

### step 2-3

site 4(N)について期待値最小化する

In [152]:
t = 4

L = L_expression(mps, O, t)
tmp_O = O[t]
H = zeros(ComplexF64, size(L, 1), size(L, 3), size(tmp_O, 1), size(tmp_O, 2))
@tensor begin
    # H[a0, a0_, sigma1, sigma2] = tmp_O[sigma1, sigma2, b0] * R[a0, b0, a0_]
    H[sigma1, a0, sigma2, a0_] = tmp_O[sigma1, sigma2, b0] * L[a0, b0, a0_]
end

H_ = zeros(ComplexF64, size(tmp_O, 1) * size(L, 1), size(tmp_O, 2) * size(L, 3))
for i=1:size(H, 1), j=1:size(H, 2), k=1:size(H, 3), l=1:size(H, 4)
    H_[(i-1)*size(H, 2) + j, (k-1)*size(H, 4) + l] = H[i,j,k,l]
end
v = eigvecs(H_)[:, 1]
λ = eigvals(H_)[1]

d = convert(Int64, size(v, 1) / 2)
mps[t][1][:] = v[1:d]
mps[t][2][:] = v[d+1:2*d]

after = expectation(mps, O)

-3.0 + 0.0im

### step 3

site 1$\to$site N の方向に 1 site ずつ更新してハミルトニアン期待値最小化

In [95]:
function left_to_right_ground_state_search(mps::Array{Any,1}, O::Array{Any,1}, D::Int64)
    hist = []
    N = size(mps, 1)
    mps_ = copy(mps)
    push!(hist, expectation(mps_, O))
    for t = 1:N
        if t == 1
            R = R_expression(mps_, O, t)
            tmp_O = O[t]
            H = zeros(ComplexF64, size(R, 1), size(R, 3), size(tmp_O, 1), size(tmp_O, 2))
            @tensor begin
                H[sigma1, a0, sigma2, a0_] = tmp_O[sigma1, sigma2, b0] * R[a0, b0, a0_]
            end
            H_ = zeros(ComplexF64, size(tmp_O, 1) * size(R, 1), size(tmp_O, 2) * size(R, 3))
            for i=1:size(H, 1), j=1:size(H, 2), k=1:size(H, 3), l=1:size(H, 4)
                H_[(i-1)*size(H, 2) + j, (k-1)*size(H, 4) + l] = H[i,j,k,l]
            end
            v = eigvecs(H_)[:, 1]
            d = convert(Int64, size(v, 1) / 2)
            mps_[t][1][:] = v[1:d]
            mps_[t][2][:] = v[d+1:2*d]
            mps_ = left_norm_for_2_sites(mps_, t, D)
            
        elseif t == N
            L = L_expression(mps_, O, t)
            tmp_O = O[t]
            H = zeros(ComplexF64, size(L, 1), size(L, 3), size(tmp_O, 1), size(tmp_O, 2))
            @tensor begin
                H[sigma1, a0, sigma2, a0_] = tmp_O[sigma1, sigma2, b0] * L[a0, b0, a0_]
            end
            H_ = zeros(ComplexF64, size(tmp_O, 1) * size(L, 1), size(tmp_O, 2) * size(L, 3))
            for i=1:size(H, 1), j=1:size(H, 2), k=1:size(H, 3), l=1:size(H, 4)
                H_[(i-1)*size(H, 2) + j, (k-1)*size(H, 4) + l] = H[i,j,k,l]
            end
            v = eigvecs(H_)[:, 1]
            d = convert(Int64, size(v, 1) / 2)
            mps_[t][1][:] = v[1:d]
            mps_[t][2][:] = v[d+1:2*d]

        else
            R = R_expression(mps_, O, t)
            L = L_expression(mps_, O, t)
            tmp_O = O[t]
            H = zeros(ComplexF64, size(tmp_O, 1), size(L, 1), size(R, 1), size(tmp_O, 2), size(L, 3), size(R, 3))
            @tensor begin
                H[sigma1, a0, a1, sigma2, a0_, a1_] = L[a0, b0, a0_] * tmp_O[sigma1, sigma2, b0, b1] * R[a1, b1, a1_]
            end
            H_ = zeros(ComplexF64, size(tmp_O, 1) * size(L, 1) * size(R, 1), size(tmp_O, 2) * size(L, 3) * size(R, 3))
            for i=1:size(H, 1), j=1:size(H, 2), k=1:size(H, 3), l=1:size(H, 4), m=1:size(H, 5), n=1:size(H, 6)
                H_[(i-1)*size(H, 2)*size(H, 3) + (j-1)*size(H, 3) + k, (l-1)*size(H, 5)*size(H, 6) + (m-1)*size(H, 6) + n] = H[i,j,k,l,m,n]
            end
            v = eigvecs(H_)[:, 1]
            d = convert(Int64, size(v, 1) / 2)
            M_1 = transpose(reshape(v[1:d], size(transpose(mps_[t][1]))))
            M_2 = transpose(reshape(v[d+1:2*d], size(transpose(mps_[t][1]))))
            mps_[t][1][:, :] = M_1
            mps_[t][2][:, :] = M_2
            mps_ = left_norm_for_2_sites(mps_, t, D)
        end
        push!(hist, expectation(mps_, O))
    end
    return (mps_, hist)
end

left_to_right_ground_state_search (generic function with 1 method)

In [96]:
D = 20
N = 4
C0 = zeros(ComplexF64, 2^N)
#C0[2] = 1 # computational basis
C0 = normalize!(rand(ComplexF64, 2^N))
mps = MPS(C0, D, 'r') # convert to MPS

O = []
push!(O, dstack((pauliZ, pauliI, pauliI)) )
push!(O, ddstack( [(pauliZ, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliI)] ))
push!(O, ddstack( [(pauliI, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliZ)] ))
push!(O, dstack((pauliI, pauliI, pauliZ)))

(opt_mps, hist) = left_to_right_ground_state_search(mps, O, D)

(Any[Array{Complex{Float64},2}[[0.0 + 0.0im -0.5607626214182947 + 0.8279766194887889im], [-0.5607626214182947 - 0.8279766194887889im 2.220446049250313e-16 + 0.0im]], Array{Complex{Float64},2}[[-0.97324245231294 + 0.2297806106178122im -1.6653345369377348e-16 + 2.220446049250313e-16im 0.0 + 0.0im 0.0 + 0.0im; 2.220446049250313e-16 + 2.3592239273284576e-16im -0.9695687216057274 - 0.2448193090502371im 0.0 + 0.0im 0.0 + 0.0im], [0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 1.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im]], Array{Complex{Float64},2}[[-1.4513420134010444e-16 + 2.516759498469181e-18im -1.7212357401774256e-15 + 2.9847798507843105e-17im; 0.0 + 0.0im 0.0 + 0.0im; 0.9996137897257442 - 0.027789771322116696im -1.6403978869705682e-16 + 1.3877787807814457e-17im; -8.500145032286355e-17 + 1.231653667943533e-16im -0.5607626214182946 + 0.8279766194887885im], [0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]], Array{Complex{Float64},2}

In [97]:
hist

5-element Array{Any,1}:
 0.006968340412331166 + 1.3010426069826053e-18im
  -1.0949265952403204 - 4.163336342344337e-17im
   -2.999999999999999 - 4.7788047400524155e-17im
  -3.0000000000000018 + 5.551115123125783e-17im
  -3.0000000000000013 + 0.0im

In [94]:
for i in 1:2^4
    println(i-1, ':', restore(opt_mps, i-1))
end

0:0.0 + 0.0im
1:0.0 + 0.0im
2:0.0 + 0.0im
3:0.0 + 0.0im
4:0.0 + 0.0im
5:0.0 + 0.0im
6:0.0 + 0.0im
7:0.0 + 0.0im
8:8.680427074014492e-17 + 1.6361324958020246e-16im
9:7.264213386742109e-32 - 2.828928674381776e-32im
10:0.8760543303935484 - 0.48221241190860276im
11:0.0 + 0.0im
12:0.0 + 0.0im
13:0.0 + 0.0im
14:0.0 + 0.0im
15:0.0 + 0.0im


### step 4

site t-1, site t に対しSVDを行い、site tをright normalizeする。

In [61]:
function right_norm_for_2_sites(mps::Array{Any,1}, t::Int64, D::Int64)
    # t: target, site to be left normalized
    mps_ = copy(mps)
    site_1 = cat(mps[t-1][1], mps[t-1][2], dims=1)
    site_2 = cat(mps[t][1], mps[t][2], dims=2)
    mixed_site = site_1 * site_2
    M, B = SVD_R(mixed_site, D)
    col = convert(Int64, size(M, 1) / 2)
    mps_[t-1] = [M[1:col, :], M[(col+1):col*2, :]]
    col2 = convert(Int64, size(B, 2) / 2)
    mps_[t] = [B[:, 1:col2], B[:, (col2+1):col2*2]]
    return mps_
end

right_norm_for_2_sites (generic function with 1 method)

操作の前後でMPSが表す状態に変化がないことを確認

In [90]:
t = 4
C0 = zeros(ComplexF64, 2^N)
C0[1] = 1 / sqrt(2) # computational basis
C0[5] = 1 / sqrt(2) # computational basis
mps = MPS(C0, D, 'r') # convert to MPS

mps_ = right_norm_for_2_sites(mps, 2, 10)
for i = 0:15
    orig = restore(mps, i)
    norm = restore(mps_, i)
    print(orig == norm, ' ')
end

true true true true true true true true true true true true true true true true 

### step 5

site N-1 $\to$ site 1 の方向(先程とは逆)に 1 site ずつ更新してハミルトニアン期待値最小化

In [102]:
function right_to_left_ground_state_search(mps::Array{Any,1}, O::Array{Any,1}, D::Int64)
    hist = []
    N = size(mps, 1)
    mps_ = copy(mps)
    push!(hist, expectation(mps_, O))
    for t = N-1:-1:1
        mps_ = right_norm_for_2_sites(mps_, t+1, D)
        if t == 1
            R = R_expression(mps_, O, t)
            tmp_O = O[t]
            H = zeros(ComplexF64, size(R, 1), size(R, 3), size(tmp_O, 1), size(tmp_O, 2))
            @tensor begin
                H[sigma1, a0, sigma2, a0_] = tmp_O[sigma1, sigma2, b0] * R[a0, b0, a0_]
            end
            H_ = zeros(ComplexF64, size(tmp_O, 1) * size(R, 1), size(tmp_O, 2) * size(R, 3))
            for i=1:size(H, 1), j=1:size(H, 2), k=1:size(H, 3), l=1:size(H, 4)
                H_[(i-1)*size(H, 2) + j, (k-1)*size(H, 4) + l] = H[i,j,k,l]
            end
            v = eigvecs(H_)[:, 1]
            d = convert(Int64, size(v, 1) / 2)
            mps_[t][1][:] = v[1:d]
            mps_[t][2][:] = v[d+1:2*d]

        else
            R = R_expression(mps_, O, t)
            L = L_expression(mps_, O, t)
            tmp_O = O[t]
            H = zeros(ComplexF64, size(tmp_O, 1), size(L, 1), size(R, 1), size(tmp_O, 2), size(L, 3), size(R, 3))
            @tensor begin
                H[sigma1, a0, a1, sigma2, a0_, a1_] = L[a0, b0, a0_] * tmp_O[sigma1, sigma2, b0, b1] * R[a1, b1, a1_]
            end
            H_ = zeros(ComplexF64, size(tmp_O, 1) * size(L, 1) * size(R, 1), size(tmp_O, 2) * size(L, 3) * size(R, 3))
            for i=1:size(H, 1), j=1:size(H, 2), k=1:size(H, 3), l=1:size(H, 4), m=1:size(H, 5), n=1:size(H, 6)
                H_[(i-1)*size(H, 2)*size(H, 3) + (j-1)*size(H, 3) + k, (l-1)*size(H, 5)*size(H, 6) + (m-1)*size(H, 6) + n] = H[i,j,k,l,m,n]
            end
            v = eigvecs(H_)[:, 1]
            d = convert(Int64, size(v, 1) / 2)
            M_1 = transpose(reshape(v[1:d], size(transpose(mps_[t][1]))))
            M_2 = transpose(reshape(v[d+1:2*d], size(transpose(mps_[t][1]))))
            mps_[t][1][:, :] = M_1
            mps_[t][2][:, :] = M_2
        end
        push!(hist, expectation(mps_, O))
    end
    return (mps_, hist)
end

right_to_left_ground_state_search (generic function with 1 method)

In [109]:
D = 20
N = 4
C0 = zeros(ComplexF64, 2^N)
#C0[2] = 1 # computational basis
C0 = normalize!(rand(ComplexF64, 2^N))
mps = MPS(C0, D, 'l') # convert to MPS

O = []
push!(O, dstack((pauliZ, pauliI, pauliI)) )
push!(O, ddstack( [(pauliZ, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliI)] ))
push!(O, ddstack( [(pauliI, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliZ)] ))
push!(O, dstack((pauliI, pauliI, pauliZ)))

(opt_mps, hist) = right_to_left_ground_state_search(mps, O, D)

(Any[Array{Complex{Float64},2}[[0.0 - 0.0im 0.0 - 0.0im], [1.0 + 0.0im -1.6820565865622263e-16 - 3.6626859291356426e-16im]], Array{Complex{Float64},2}[[-3.3856405763623705e-16 + 3.1786605777263444e-17im 3.122502256758253e-17 - 1.3877787807814457e-17im 0.7835107529064149 - 0.16737397362303652im 0.5852082781884836 - 0.12501249606870588im; -2.9375070561846326e-16 + 4.143549165033361e-17im -0.3073569134261788 - 0.006411949689282673im 0.16030019460140327 - 0.5464038681924793im -0.21461918917479506 + 0.73155716027069im], [0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im]], Array{Complex{Float64},2}[[0.8011885838966643 + 0.0im -0.598411942589432 - 6.52323148448189e-17im; -0.598411942589432 + 0.0im -0.8011885838966643 - 8.733680302012205e-17im; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im], [0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im; 1.0 + 0.0im 0.0 + 0.0im]], Array{Complex{Float64},2}[[-0.5984119425894324 + 0.

In [110]:
hist

4-element Array{Any,1}:
 -0.13283384523121178 - 5.551115123125783e-17im
  -3.0000000000000036 + 2.7755575615628914e-17im
  -3.0000000000000018 - 1.5223152443483392e-18im
  -3.0000000000000004 - 1.4319233930506864e-19im

In [111]:
for i in 1:2^4
    println(i-1, ':', restore(opt_mps, i-1))
end

0:0.0 + 0.0im
1:8.057569884245053e-17 + 9.977592261580458e-17im
2:0.0 + 0.0im
3:0.0 + 0.0im
4:0.0 + 0.0im
5:-0.9779354931590871 + 0.2089070875286263im
6:0.0 + 0.0im
7:0.0 + 0.0im
8:0.0 + 0.0im
9:3.3637685397118943e-16 - 4.9866035960335757e-17im
10:0.0 + 0.0im
11:0.0 + 0.0im
12:0.0 + 0.0im
13:-1.9193318606758697e-16 + 1.1250151085496812e-16im
14:0.0 + 0.0im
15:0.0 + 0.0im


### step 6

上記の関数を統合して、site 1 $\to$ site N $\to$ site1 の1往復でground state searchを行う

In [112]:
function left_most_site_update(mps_::Array{Any,1}, O::Array{Any,1}, t::Int64)
    R = R_expression(mps_, O, t)
    tmp_O = O[t]
    H = zeros(ComplexF64, size(R, 1), size(R, 3), size(tmp_O, 1), size(tmp_O, 2))
    @tensor begin
        H[sigma1, a0, sigma2, a0_] = tmp_O[sigma1, sigma2, b0] * R[a0, b0, a0_]
    end
    H_ = zeros(ComplexF64, size(tmp_O, 1) * size(R, 1), size(tmp_O, 2) * size(R, 3))
    for i=1:size(H, 1), j=1:size(H, 2), k=1:size(H, 3), l=1:size(H, 4)
        H_[(i-1)*size(H, 2) + j, (k-1)*size(H, 4) + l] = H[i,j,k,l]
    end
    v = eigvecs(H_)[:, 1]
    d = convert(Int64, size(v, 1) / 2)
    mps_[t][1][:] = v[1:d]
    mps_[t][2][:] = v[d+1:2*d]
    mps_ = left_norm_for_2_sites(mps_, t, D)
    return mps_
end

function mid_site_update(mps_::Array{Any,1}, O::Array{Any,1}, t::Int64)
    R = R_expression(mps_, O, t)
    L = L_expression(mps_, O, t)
    tmp_O = O[t]
    H = zeros(ComplexF64, size(tmp_O, 1), size(L, 1), size(R, 1), size(tmp_O, 2), size(L, 3), size(R, 3))
    @tensor begin
        H[sigma1, a0, a1, sigma2, a0_, a1_] = L[a0, b0, a0_] * tmp_O[sigma1, sigma2, b0, b1] * R[a1, b1, a1_]
    end
    H_ = zeros(ComplexF64, size(tmp_O, 1) * size(L, 1) * size(R, 1), size(tmp_O, 2) * size(L, 3) * size(R, 3))
    for i=1:size(H, 1), j=1:size(H, 2), k=1:size(H, 3), l=1:size(H, 4), m=1:size(H, 5), n=1:size(H, 6)
        H_[(i-1)*size(H, 2)*size(H, 3) + (j-1)*size(H, 3) + k, (l-1)*size(H, 5)*size(H, 6) + (m-1)*size(H, 6) + n] = H[i,j,k,l,m,n]
    end
    v = eigvecs(H_)[:, 1]
    d = convert(Int64, size(v, 1) / 2)
    M_1 = transpose(reshape(v[1:d], size(transpose(mps_[t][1]))))
    M_2 = transpose(reshape(v[d+1:2*d], size(transpose(mps_[t][1]))))
    mps_[t][1][:, :] = M_1
    mps_[t][2][:, :] = M_2
    return mps_
end
    
function right_most_site_update(mps_::Array{Any,1}, O::Array{Any,1}, t::Int64)
    L = L_expression(mps_, O, t)
    tmp_O = O[t]
    H = zeros(ComplexF64, size(L, 1), size(L, 3), size(tmp_O, 1), size(tmp_O, 2))
    @tensor begin
        H[sigma1, a0, sigma2, a0_] = tmp_O[sigma1, sigma2, b0] * L[a0, b0, a0_]
    end
    H_ = zeros(ComplexF64, size(tmp_O, 1) * size(L, 1), size(tmp_O, 2) * size(L, 3))
    for i=1:size(H, 1), j=1:size(H, 2), k=1:size(H, 3), l=1:size(H, 4)
        H_[(i-1)*size(H, 2) + j, (k-1)*size(H, 4) + l] = H[i,j,k,l]
    end
    v = eigvecs(H_)[:, 1]
    d = convert(Int64, size(v, 1) / 2)
    mps_[t][1][:] = v[1:d]
    mps_[t][2][:] = v[d+1:2*d]
    return mps_
end

right_most_site_update (generic function with 1 method)

In [126]:
function iterative_ground_state_search(mps::Array{Any,1}, O::Array{Any,1}, D::Int64)
    hist = []
    N = size(mps, 1)
    mps_ = copy(mps)
    push!(hist, expectation(mps_, O))
    for t = 1:N
        if t == 1
            mps_ = left_most_site_update(mps_, O, t)
            mps_ = left_norm_for_2_sites(mps_, t, D)  
        elseif t == N
            mps_ = right_most_site_update(mps_, O, t)
        else
            mps_ = mid_site_update(mps_, O, t)
            mps_ = left_norm_for_2_sites(mps_, t, D)
        end
        push!(hist, expectation(mps_, O))
    end
    
    for t = N-1:-1:1
        mps_ = right_norm_for_2_sites(mps_, t+1, D)
        if t == 1
            mps_ = left_most_site_update(mps_, O, t)
        else
            mps_ = mid_site_update(mps_, O, t)
        end  
        push!(hist, expectation(mps_, O))
    end
    return (mps_, hist)
end

iterative_ground_state_search (generic function with 1 method)

In [11]:
D = 20
N = 4
C0 = zeros(ComplexF64, 2^N)
#C0[2] = 1 # computational basis
C0 = normalize!(rand(ComplexF64, 2^N))
mps = MPS(C0, D, 'l') # convert to MPS

O = []
push!(O, dstack((pauliZ, pauliI, pauliI)) )
push!(O, ddstack( [(pauliZ, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliI)] ))
push!(O, ddstack( [(pauliI, zero, zero), (zero, pauliZ, zero), (zero, zero, pauliZ)] ))
push!(O, dstack((pauliI, pauliI, pauliZ)))

(opt_mps, hist) = iterative_ground_state_search(mps, O, D, 2)

(Any[Array{Complex{Float64},2}[[0.0 + 0.0im 0.0 + 0.0im], [1.0 + 0.0im 0.0 + 0.0im]], Array{Complex{Float64},2}[[0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im], [0.020203972330057347 - 0.009450387301157982im -2.408253154218805e-16 - 2.2089218404179512e-16im 0.0020264553518257607 - 2.8952502976456115e-10im 0.999749160120003 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im]], Array{Complex{Float64},2}[[1.3449787562838703e-16 - 7.295598192220656e-19im 0.986935449912459 - 0.16111616192868988im … -1.2118612012662745e-15 + 4.928088509102355e-16im -6.099149198453333e-6 - 5.8532879798838705e-6im; -7.48591581876233e-27 + 6.992001794320268e-28im 4.647883664731597e-19 - 2.3211484642396814e-19im … 4.2583118208009745e-20 - 1.9957137379614044e-20im 1.834725161020029e-16 + 1.2462248542428e-16im; 3.224981229778437e-14 - 1.7493337437480885e-16im -1.691091676577086e-8 + 2.7606893436863886e-9im … 0.000373982371997314 - 5.152807586576629e-5im -0.0

In [12]:
hist

17-element Array{Any,1}:
 -0.015370730768370469 - 1.734723475976807e-17im
    -2.999999999999999 + 5.5511151231257796e-17im
    -2.999999999999999 + 4.2847669856627135e-16im
    -2.999999999999999 + 5.3231195397338055e-18im
    -3.000000000000001 - 1.0798224414970604e-22im
    -3.000000000000001 + 3.3828027494849346e-17im
    -3.000000000000001 + 5.334482184232308e-16im
   -3.0000000000000013 + 5.4907713889351585e-16im
   -3.0000000000000013 + 5.4907713889351585e-16im
   -3.0000000000000027 - 3.764516576578242e-31im
   -3.0000000000000036 + 1.726049858596923e-16im
   -3.0000000000000027 - 1.1188966420050406e-16im
   -3.0000000000000036 + 1.6629269634713695e-16im
   -3.0000000000000027 - 1.5960474149342653e-16im
    -3.000000000000001 - 1.66696140184227e-16im
   -3.0000000000000027 + 1.72959145291531e-16im
   -3.0000000000000027 + 1.72959145291531e-16im

In [132]:
for i in 1:2^4
    println(i-1, ':', restore(opt_mps, i-1))
end

0:-1.1851219930960202e-32 - 2.4768127144321423e-33im
1:-4.9727278965174063e-17 - 7.121600858597644e-18im
2:0.0 + 0.0im
3:0.0 + 0.0im
4:-1.906831226944259e-48 - 8.503529943549105e-49im
5:-0.9999341849601532 - 0.011472826507641004im
6:0.0 + 0.0im
7:0.0 + 0.0im
8:0.0 + 0.0im
9:0.0 + 0.0im
10:0.0 + 0.0im
11:0.0 + 0.0im
12:-1.6016765293919356e-33 - 1.666643679072051e-33im
13:-7.072805261199743e-18 - 6.477488609716518e-18im
14:0.0 + 0.0im
15:0.0 + 0.0im


In [15]:
D = 10
N = 9
C0 = zeros(ComplexF64, 2^N)
#C0[2] = 1 # computational basis
C0 = normalize!(rand(ComplexF64, 2^N))
mps = MPS(C0, D, 'r') # convert to MPS

O = []
push!(O, dstack((zero, pauliZ, pauliI)) )
for i in 1:(N-2)
    push!(O, ddstack( [(pauliI, zero, zero), (pauliZ, zero, zero), (zero, pauliZ, pauliI)] ))
end
push!(O, dstack((pauliI, pauliZ, zero)))

(opt_mps, hist) = iterative_ground_state_search(mps, O, D, 2)

(Any[Array{Complex{Float64},2}[[0.0 + 0.0im 0.0 + 0.0im], [1.0 + 0.0im 0.0 + 0.0im]], Array{Complex{Float64},2}[[-1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im], [0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im]], Array{Complex{Float64},2}[[0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im], [1.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im]], Array{Complex{Float64},2}[[-1.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im], [0.0 + 0.0

In [16]:
hist

17-element Array{Any,1}:
 0.1330780736654148 - 1.8973538018496328e-17im
 -8.000000000000005 - 1.1102230246251565e-16im
 -8.000000000000005 - 4.440892098500626e-16im
               -8.0 - 4.440892098500626e-16im
               -8.0 - 5.551115123125783e-16im
 -8.000000000000004 - 1.0408340855860843e-16im
 -8.000000000000007 + 0.0im
 -8.000000000000007 + 0.0im
 -8.000000000000007 + 0.0im
               -8.0 + 3.9443045261050222e-31im
 -7.999999999999998 + 1.5777218104420236e-30im
 -7.999999999999999 + 0.0im
 -7.999999999999999 - 2.1895288505075236e-46im
 -8.000000000000005 - 4.379057701015084e-47im
 -8.000000000000005 - 4.379057701015084e-47im
 -8.000000000000005 - 4.379057701015084e-47im
 -8.000000000000005 - 4.379057701015084e-47im